1) import log data <br/>
2) calculate Vp and Vs from DTCO and DTSM in m/s from microsec/ft <br/>
3) plot vs Depth <br/>
4) edit the high anomaly peak by mean or remove them

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import statistics as stat
from scipy import stats

#from matplotlib.ticker import FuncFormatter # for custom number format on x and y axis
#from io import StringIO
#from scipy import stats
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
pd.set_option('display.max_rows', 10)

In [ ]:
st = pd.read_excel('SampleData/logdataVpVs.xlsx') #use sample# as index column
st

In [ ]:
#st.info()

In [ ]:
st['Vp(m_s)']= (1/ st['DTCO'])* (0.3048/ 10**-6)
st['Vs(m_s)']= (1/ st['DTSM'])* (0.3048/ 10**-6)
st

In [ ]:
#st.info()

In [ ]:
fig, ax = plt.subplots(1,5,sharey=True)
fig.set_size_inches(10,10)

plt.sca(ax[0])
plt.plot(st['DTCO'], st['DEPTH'])
plt.grid()
ax[0].invert_yaxis()
plt.ylabel('Depth(m)')
plt.xlabel('DTCO(microsec/ft)')

    
plt.sca(ax[1])
plt.plot(st['DTSM'], st['DEPTH'])
plt.grid()
plt.xlabel('DTSM(microsec/ft)')
    
plt.sca(ax[2])
plt.plot(st['RHOZ'], st['DEPTH'])
plt.grid()
plt.xlabel('RHOZ(g/cm3)')

plt.sca(ax[3])
plt.plot(st['Vp(m_s)'], st['DEPTH'])
plt.grid()
plt.xlabel('Vp(m/s)')
    
plt.sca(ax[4])
plt.plot(st['Vs(m_s)'], st['DEPTH'])
plt.grid()
plt.xlabel('Vs(m/s)')

In [ ]:
st_new = pd.DataFrame()
st_new['DEPTH']= st['DEPTH']
st_new['DTCO']= st['DTCO']
st_new['DTSM']= st['DTSM']
st_new['RHOZ']= st['RHOZ']
st_new.head()

In [ ]:
meanDTCO= stat.mean(st['DTCO'])
meanDTSM= stat.mean(st['DTSM'])
meanRHOZ= stat.mean(st['RHOZ'])

In [ ]:
# create def for correcting data
def correct_DTCO(df):
    if df['DTCO'] < -1000:
        return meanDTCO
    else:
        return df['DTCO']
    
def correct_DTSM(df):
    if df['DTSM'] < -1000:
        return meanDTSM
    else:
        return df['DTSM']
    
def correct_RHOZ(df):
    if df['RHOZ'] < -1000:
        return meanRHOZ
    else:
        return df['RHOZ']

In [ ]:
# create new columns
st_new['DEPTH_new']= st['DEPTH']
st_new['DTCO_new']= st_new.apply(correct_DTCO, axis=1)
st_new['DTSM_new']= st_new.apply(correct_DTSM, axis=1)
st_new['RHOZ_new']= st_new.apply(correct_RHOZ, axis=1)
st_new

In [ ]:
st_new['Vp_new(m_s)']= (1/ st_new['DTCO_new'])* (0.3048/ 10**-6)
st_new['Vs_new(m_s)']= (1/ st_new['DTSM_new'])* (0.3048/ 10**-6)
st_new

In [ ]:
fig, ax_1 = plt.subplots(1,5,sharey=True)
fig.set_size_inches(10,10)

plt.sca(ax_1[0])
plt.plot(st_new['DTCO_new'], st_new['DEPTH_new'])
plt.grid()
ax_1[0].invert_yaxis()
plt.ylabel('Depth(m)')
plt.xlabel('DTCO(microsec/ft)')

    
plt.sca(ax_1[1])
plt.plot(st_new['DTSM_new'], st_new['DEPTH_new'])
plt.grid()
plt.xlabel('DTSM(microsec/ft)')
    
plt.sca(ax_1[2])
plt.plot(st_new['RHOZ_new'], st_new['DEPTH_new'])
plt.grid()
plt.xlabel('RHOZ(g/cm3)')

plt.sca(ax_1[3])
plt.plot(st_new['Vp_new(m_s)'], st_new['DEPTH_new'])
plt.grid()
plt.xlabel('Vp(m/s)')
    
plt.sca(ax_1[4])
plt.plot(st_new['Vs_new(m_s)'], st_new['DEPTH_new'])
plt.grid()
plt.xlabel('Vs(m/s)')

In [ ]:
st2= st

In [ ]:
mean1, sigma1 = st['DTCO'].mean(), st['DTCO'].std()
print(mean1, sigma1)

mean2, sigma2 = st['DTSM'].mean(), st['DTSM'].std()
print(mean2, sigma2)

mean3, sigma3 = st['RHOZ'].mean(), st['RHOZ'].std()
print(mean3, sigma3)

In [ ]:
conf_int_DTCO = stats.norm.interval(0.98, loc=mean1, scale=sigma1)
print(conf_int_DTCO[0])
print(conf_int_DTCO[1])

conf_int_DTSM = stats.norm.interval(0.98, loc=mean2, scale=sigma2)
print(conf_int_DTSM[0])
print(conf_int_DTSM[1])

conf_int_RHOZ = stats.norm.interval(0.98, loc=mean3, scale=sigma3)
print(conf_int_RHOZ[0])
print(conf_int_RHOZ[1])

In [ ]:
indDTCO_l= st2.index[st2['DTCO'] < conf_int_DTCO[0]].tolist()
indDTCO_h= st2.index[st2['DTCO'] > conf_int_DTCO[1]].tolist()

indDTSM_l= st2.index[st2['DTSM'] < conf_int_DTSM[0]].tolist()
indDTSM_h= st2.index[st2['DTSM'] > conf_int_DTSM[1]].tolist()

indRHOZ_l= st2.index[st2['RHOZ'] < conf_int_RHOZ[0]].tolist()
indRHOZ_h= st2.index[st2['RHOZ'] > conf_int_RHOZ[1]].tolist()

u_ind= indDTCO_l+ indDTCO_h+ indDTSM_l+ indDTSM_h+ indRHOZ_l+ indRHOZ_h
u_ind

In [ ]:
st2= st2.drop(u_ind)
st2

In [ ]:
st2['Vp(m_s)']= (1/ st2['DTCO'])* (0.3048/ 10**-6)
st2['Vs(m_s)']= (1/ st2['DTSM'])* (0.3048/ 10**-6)
st2

In [ ]:
fig, ax_1 = plt.subplots(1,5,sharey=True)
fig.set_size_inches(10,10)

plt.sca(ax_1[0])
plt.plot(st2['DTCO'], st2['DEPTH'])
plt.grid()
ax_1[0].invert_yaxis()
plt.ylabel('Depth(m)')
plt.xlabel('DTCO(microsec/ft)')

    
plt.sca(ax_1[1])
plt.plot(st2['DTSM'], st2['DEPTH'])
plt.grid()
plt.xlabel('DTSM(microsec/ft)')
    
plt.sca(ax_1[2])
plt.plot(st2['RHOZ'], st2['DEPTH'])
plt.grid()
plt.xlabel('RHOZ(g/cm3)')

plt.sca(ax_1[3])
plt.plot(st2['Vp(m_s)'], st2['DEPTH'])
plt.grid()
plt.xlabel('Vp(m/s)')
    
plt.sca(ax_1[4])
plt.plot(st2['Vs(m_s)'], st2['DEPTH'])
plt.grid()
plt.xlabel('Vs(m/s)')